In [2]:
import pandas as pd
import numpy as np
import torch
import transformers

d:\anaconda\envs\ml\Lib\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
data = pd.read_csv('archive/Womens Clothing E-Commerce Reviews.csv')
data.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [20]:
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')

model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')

text = ['I love your dress and i very like it and i love you']

inputs = tokenizer(text, padding='max_length', max_length=75, truncation=True, return_tensors='pt')
inputs


{'input_ids': tensor([[ 101, 1045, 2293, 2115, 4377, 1998, 1045, 2200, 2066, 2009, 1998, 1045,
         2293, 2017,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        

In [21]:
outputs = model(**inputs)
outputs


SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1665, -2.0732,  1.9908]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [6]:
data['Recommended IND'].value_counts()

Recommended IND
1    19314
0     4172
Name: count, dtype: int64

In [5]:
dataset = data[['Review Text', 'Recommended IND']]

In [70]:
dataset

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
23481,I was very happy to snag this dress at such a ...,1
23482,"It reminds me of maternity clothes. soft, stre...",1
23483,"This fit well, but the top was very see throug...",0
23484,I bought this dress for a wedding i have this ...,1


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
russian_stop_words = stopwords.words('russian')
    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
import spacy

nlp = spacy.load('ru_core_news_sm')

doc = nlp('Программирование это моя и его жизнь')

lemma = [token.lemma_ for token in doc]

lemma


['программирование', 'это', 'мой', 'и', 'он', 'жизнь']

In [8]:
dataset = dataset.dropna()

In [72]:
dataset.shape

(22641, 2)

In [9]:
from tqdm.notebook import tqdm
from tqdm import tqdm

def process_text(text):
    text = text.lower()
    
    text = re.sub(r'[^\w\s]', '', text)
    
    text = re.findall(r'\S+|\s+', text)    
    
    cleared_words = []

    for word in text:
        if word.isspace():
            cleared_words.append(word)
        else:
            doc = nlp(word)
            lemma = doc[0].lemma_
            
            if word not in russian_stop_words:
                
                cleared_words.append(lemma)
            else:
                cleared_words.append(word)
            
        
    return ''.join(cleared_words)
        
            

In [73]:
tqdm.pandas()
clean_dataset = dataset.copy()
clean_dataset['Review Text'] = dataset['Review Text'].progress_apply(process_text)

100%|██████████| 22641/22641 [2:11:57<00:00,  2.86it/s]  


In [74]:
clean_dataset

,Review Text,Recommended IND
0,absolutely wonderful silky and sexy and comfo...,1
1,love this dress its sooo pretty i happened t...,1
2,i had such high hopes for this dress and reall...,0
3,i love love love this jumpsuit its fun flirty ...,1
4,this shirt is very flattering to all due to th...,1
...,...,...
23481,i was very happy to snag this dress at such a ...,1
23482,it reminds me of maternity clothes soft stretc...,1
23483,this fit well but the top was very see through...,0
23484,i bought this dress for a wedding i have this ...,1


In [78]:
clean_dataset.to_csv('clean_womens_clothing.csv', index=False)

In [51]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [84]:
clean_dataset

,Review Text,Recommended IND
0,absolutely wonderful silky and sexy and comfo...,1
1,love this dress its sooo pretty i happened t...,1
2,i had such high hopes for this dress and reall...,0
3,i love love love this jumpsuit its fun flirty ...,1
4,this shirt is very flattering to all due to th...,1
...,...,...
23481,i was very happy to snag this dress at such a ...,1
23482,it reminds me of maternity clothes soft stretc...,1
23483,this fit well but the top was very see through...,0
23484,i bought this dress for a wedding i have this ...,1


In [10]:
from datasets import load_dataset
from transformers import AdamW, get_scheduler

full_dataset = load_dataset('csv', data_files='clean_womens_clothing.csv')

full_dataset = full_dataset.filter(lambda x : x['Review Text'] is not None and x['Recommended IND'] is not None)

full_dataset

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Recommended IND'],
        num_rows: 22641
    })
})

In [88]:
full_dataset['train'][1]['Review Text']

'love this dress  its sooo pretty  i happened to find it in a store and im glad i did bc i never would have ordered it online bc its petite  i bought a petite and am 58  i love the length on me hits just a little below the knee  would definitely be a true midi on someone who is truly petite'

In [11]:
data = full_dataset['train'].train_test_split(test_size=0.2)
data

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Recommended IND'],
        num_rows: 18112
    })
    test: Dataset({
        features: ['Review Text', 'Recommended IND'],
        num_rows: 4529
    })
})

In [12]:
data = data.map(lambda x : tokenizer(x['Review Text'], padding='max_length', max_length=75, truncation=True), batched=True)
    
    

Map:   0%|          | 0/18112 [00:00<?, ? examples/s]

Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

In [13]:
data

DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Recommended IND', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 18112
    })
    test: Dataset({
        features: ['Review Text', 'Recommended IND', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4529
    })
})

In [14]:
data = data.remove_columns('Review Text')

In [22]:
data

DatasetDict({
    train: Dataset({
        features: ['Recommended IND', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 18112
    })
    test: Dataset({
        features: ['Recommended IND', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4529
    })
})

In [27]:
data['train']['Recommended IND']

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [15]:
import torch
from torch.utils.data import DataLoader

train_dataloader = DataLoader(data['train'], shuffle=True, batch_size=16)
test_dataloader = DataLoader(data['test'], shuffle=False, batch_size=8)

In [28]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(),lr=1e-4,)

num_epochs = 3
num_train_steps = num_epochs * len(train_dataloader)

lr_sheduler = get_scheduler(
    'cosine',
    optimizer=optimizer,
    num_training_steps= num_train_steps,
    num_warmup_steps= int(0.1 * num_epochs * len(train_dataloader))
)

def add_labels(dataset):
    dataset = dataset.map(lambda x: {'labels': x['Recommended IND']})
    return dataset

data['train'] = add_labels(data['train'])
data['test'] = add_labels(data['test'])


d:\anaconda\envs\ml\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Map:   0%|          | 0/18112 [00:00<?, ? examples/s]

Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

In [30]:
data = data.remove_columns('Recommended IND')

In [31]:
data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 18112
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4529
    })
})

In [34]:
from transformers import Trainer, TrainingArguments
from evaluate import load

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model.to(device)

f1 = load('f1')

training_args = TrainingArguments(
    output_dir='./result',
    num_train_epochs=num_epochs,
    weight_decay=0.02,
    warmup_steps=400,
    logging_dir='./runs',
    logging_strategy='epoch',
    save_strategy='best',
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    report_to='none'
)

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return f1.compute(predictions=predictions, references=labels, average='binary')

trainer = Trainer(
    model=model,
    train_dataset=data['train'],
    eval_dataset=data['test'],
    args=training_args,
    compute_metrics=compute_metrics
    
)

cuda


d:\anaconda\envs\ml\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [35]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.212700,0.442888,0.929688
2,0.188300,0.423397,0.943059
3,0.080700,0.490136,0.945834


TrainOutput(global_step=6792, training_loss=0.16056794749553127, metrics={'train_runtime': 2264.9353, 'train_samples_per_second': 23.99, 'train_steps_per_second': 2.999, 'total_flos': 2094218359228800.0, 'train_loss': 0.16056794749553127, 'epoch': 3.0})